In [31]:
!pip install cassandra-driver -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 984.2 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: cassandra-driver
    Found existing installation: cassandra-driver 3.28.0
    Uninstalling cassandra-driver-3.28.0:
      Successfully uninstalled cassandra-driver-3.28.0

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
from dotenv import load_dotenv
from conn import getCQLSession
from cassandra.cqlengine.query import BatchStatement
from cassandra.query import BatchType
import csv

load_dotenv( override=True) 
os.environ["MODE"]

'astra_db'

In [7]:
from conn import getCQLSession
session = getCQLSession(os.environ["MODE"])

Initializing CQL Session
Connected


In [8]:
table = "default_keyspace.product_catalog_emb"

session.execute("""CREATE TABLE IF NOT EXISTS default_keyspace.product_catalog_emb (
                    id text PRIMARY KEY,
                    title text,
                    emb VECTOR<FLOAT, 384>)""")

session.execute("""CREATE CUSTOM INDEX product_catalog_emb_vector_idx
    ON default_keyspace.product_catalog_emb (emb)
    USING 'StorageAttachedIndex'  
    WITH OPTIONS = { 'similarity_function': 'COSINE'};""")

CREATE CUSTOM INDEX product_catalog_emb_vector_idx
    ON default_keyspace.product_catalog_emb (emb)
    USING 'StorageAttachedIndex'  
    WITH OPTIONS = { 'similarity_function': 'COSINE'};

CREATE CUSTOM INDEX product_catalog_emb_vector_idx
    ON default_keyspace.product_catalog_emb (emb)
    USING 'StorageAttachedIndex'  
    WITH OPTIONS = { 'source_model': 'bert', 'similarity_function': 'COSINE'};
    
CREATE CUSTOM INDEX product_catalog_emb_vector_idx
    ON default_keyspace.product_catalog_emb (emb)
    USING 'StorageAttachedIndex'  
    WITH OPTIONS = { 'source_model': 'bert', 'similarity_function': 'DOT_PRODUCT'};    

InvalidRequest: Error from server: code=2200 [Invalid query] message="Index 'product_catalog_emb_vector_idx' already exists"

In [15]:
cmd_insert = f"""
INSERT INTO {table} (id, title, emb)
VALUES (:id, :title, :emb)
"""

prepared_stmt_insert = session.prepare(cmd_insert)

In [10]:
def read_csv(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            data.append(row)
    return data

In [39]:
def save_data(data, batch_size = 20):
    batch = BatchStatement(batch_type=BatchType.UNLOGGED)
    count = 0
    for row in data:
        batch.add(prepared_stmt_insert, {"id": row['asin'], "title": row['title'], "emb": [float(x) for x in row['emb'].split(',')] })
        count += 1
        if count % batch_size == 0:
            rs = session.execute(batch)
            batch.clear()

        if count % 1000 == 0 :
            print(f"""{count} records inserted.""")
    
    
    rs = session.execute(batch)
    batch.clear()
    
    return count

In [12]:
data = read_csv("../data_embeddings/chunk_9999.csv")

In [29]:
len(data)

10000

In [27]:
len([float(x) for x in data[0]['emb'].split(',')])

384

In [53]:
import time
start = time.time()
save_data(data,100)
end = time.time()
print(end - start)

1000 records inserted.
2000 records inserted.
3000 records inserted.
4000 records inserted.
5000 records inserted.
6000 records inserted.
7000 records inserted.
8000 records inserted.
9000 records inserted.
10000 records inserted.
77.29538297653198


In [55]:
len(data) / (end - start)

129.37383340265157

# Product Search

In [58]:
queries = read_csv("../data_queries/chunk_9999.csv")

In [59]:
queries[0]

{'query': 'trellis for climbing plants',
 'emb': '-0.048838172,-0.074733235,0.05481011,0.027351009,0.0006103623,0.026222544,-0.008075648,0.0138345035,-0.0077102883,0.03429652,0.009751471,-0.05707372,0.045820367,0.025954433,0.05993116,0.056023695,0.011463513,0.10386538,-0.048460662,-0.018406807,0.04325874,0.012444333,-0.030416965,0.0017623174,-0.0153835965,0.011319403,-0.08990371,-0.005596035,-0.014435396,-0.11805417,0.011254357,0.035645973,0.01903037,0.006994528,-0.010007574,0.0066225445,-0.053514514,-0.0013567668,-0.034658812,0.017996404,0.013226245,0.0060074404,-0.07308237,-0.001998519,-0.055668496,-0.014055906,-0.027527863,-0.014177166,0.030189157,-0.041558646,-0.033268902,-0.050733875,-0.013386321,0.001383755,-0.020400908,0.0132677,0.036865756,0.059189137,0.010553816,0.02062063,0.08669635,-0.034591265,-0.11797153,0.0666018,0.02127234,0.0060160155,-0.045337945,0.027114328,0.08377424,0.07520063,0.06897177,0.015983732,0.007868892,0.006487555,-0.01724752,-0.024720518,-0.037986327,-0.04

In [106]:
import time

def query_data(data, limit="10"):
    cmd_select = f"""
        SELECT id, title FROM {table}
        ORDER BY emb ANN OF :emb LIMIT {limit}
    """

    prepared_stmt_select = session.prepare(cmd_select)
    count = 0
    start = time.time()
    for row in data:
        count += 1
        prods = session.execute(prepared_stmt_select,{"emb": [float(x) for x in row['emb'].split(',')] })
        if count % 10 == 0 :
            end = time.time()
            print(f"""{count} queries | Time: {end - start} | RPS: {(end - start) / 10} """)
            start = time.time()
    
    print(f"""Finish: {count} queries.""")
    return count

In [116]:
# source_model="bert", limit 10, 
query_data(queries, limit="1000")

10 queries | Time: 14.927029132843018 | RPS: 1.4927029132843017 
20 queries | Time: 11.99948000907898 | RPS: 1.199948000907898 
30 queries | Time: 11.287514925003052 | RPS: 1.1287514925003053 
40 queries | Time: 9.101099967956543 | RPS: 0.9101099967956543 
50 queries | Time: 9.797742128372192 | RPS: 0.9797742128372192 
60 queries | Time: 10.387034177780151 | RPS: 1.0387034177780152 


KeyboardInterrupt: 

In [115]:
# source_model="bert", DOT_PRODUCT metric 
query_data(queries, limit="10")

10 queries | Time: 2.7655727863311768 | RPS: 0.2765572786331177 
20 queries | Time: 1.916346788406372 | RPS: 0.1916346788406372 
30 queries | Time: 1.8731200695037842 | RPS: 0.18731200695037842 
40 queries | Time: 1.8521580696105957 | RPS: 0.18521580696105958 
50 queries | Time: 1.7207322120666504 | RPS: 0.17207322120666504 
60 queries | Time: 1.708362102508545 | RPS: 0.1708362102508545 
70 queries | Time: 1.6757018566131592 | RPS: 0.16757018566131593 
80 queries | Time: 1.687256097793579 | RPS: 0.1687256097793579 
90 queries | Time: 1.754384994506836 | RPS: 0.1754384994506836 
100 queries | Time: 1.6938869953155518 | RPS: 0.16938869953155516 
110 queries | Time: 1.7073028087615967 | RPS: 0.17073028087615966 


KeyboardInterrupt: 

In [ ]:
cmd_select = f"""
    SELECT id, title FROM {table}
    ORDER BY emb ANN OF :emb LIMIT {limit}
"""

prepared_stmt_select = session.prepare(cmd_select)
prods = session.execute(prepared_stmt_select,{"emb": [float(x) for x in data[0]['emb'].split(',')] })



In [72]:
# Limit 10, No source_model, COSINE
query_data(queries)

10 queries | Time: 10.855203628540039 | RPS: 1.085520362854004 
20 queries | Time: 15.211241960525513 | RPS: 1.5211241960525512 
30 queries | Time: 20.742655038833618 | RPS: 2.0742655038833617 
40 queries | Time: 16.884514093399048 | RPS: 1.6884514093399048 
50 queries | Time: 16.73455500602722 | RPS: 1.673455500602722 
60 queries | Time: 17.008038759231567 | RPS: 1.7008038759231567 
70 queries | Time: 13.05151104927063 | RPS: 1.305151104927063 
80 queries | Time: 15.704714059829712 | RPS: 1.5704714059829712 
90 queries | Time: 12.193763017654419 | RPS: 1.219376301765442 
100 queries | Time: 12.8961181640625 | RPS: 1.28961181640625 
110 queries | Time: 13.630681991577148 | RPS: 1.363068199157715 
120 queries | Time: 14.791709899902344 | RPS: 1.4791709899902343 
130 queries | Time: 12.616060018539429 | RPS: 1.2616060018539428 
140 queries | Time: 11.029661893844604 | RPS: 1.1029661893844604 
150 queries | Time: 10.504261016845703 | RPS: 1.0504261016845704 
160 queries | Time: 9.07820701

KeyboardInterrupt: 

In [114]:
%time
cmd_select = f"""
        SELECT id FROM default_keyspace.product_catalog_emb
        ORDER BY emb ANN OF :emb LIMIT 1000
"""
start = time.time()
prepared_stmt_select = session.prepare(cmd_select)
prods = session.execute(prepared_stmt_select,{"emb": [float(x) for x in queries[2]['emb'].split(',')] })
end = time.time()
print(f"""Time: {end - start}""")
df = pd.DataFrame(prods)
df

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 1.91 µs
Time: 1.9042670726776123


,id
0,B072WCKPSH
1,B06WGPCNG4
2,B0BF8WS1CY
3,B0C1Z8F1L9
4,B0BG51NTTB
...,...
995,B07CQBLG7K
996,B0BRPY6KCY
997,B0BRPZ1DN4
998,B09W43SM25
